In [1]:
%load_ext autoreload
%autoreload 2
import datetime
from glob import glob
import os
from pathlib import Path
import math
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from train_prophet import main_prophet
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [30]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if 
               path.count('DT') or path.count('day_no') or path.count('Amt')
               or path.count('ID')
               or path.count('card')
               or path.count('add')
               or path.count('dist')
               or path.count('dist')
               or path.count('mail')
              ]
test_paths  = [path for path in test_paths  
               if path.count('DT') or path.count('day_no') or path.count('Amt')
               or path.count('ID')
               or path.count('card')
               or path.count('add')
               or path.count('dist')
               or path.count('dist')
               or path.count('mail')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)
event_df = pd.read_csv('../input/usa_event_calendar.csv')
event_df['date'] = event_df['date'].map(lambda x: datetime.date(*[int(d) for d in x.split('-')]))
event_df.set_index('date', inplace=True)

In [31]:
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [26]:
indicator = 'TransactionAmt'
data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
data['month'] = data['datetime'].map(lambda x: x.month)
data['date'] = data['datetime'].map(lambda x: x.date())
key_cols = [col for col in data.columns if not col.count('Trans') and col != 'datetime' and not col.count('dist')]
viz = data.groupby(key_cols)[indicator].sum()
viz = viz.to_frame(indicator + '_sum').reset_index()
viz.to_csv('../eda/eda_start_date_by_some_category.csv', index=True)

In [41]:
# train_iden = pd.read_csv('../input/train_identity.csv')
# test_iden = pd.read_csv('../input/test_identity.csv')
# iden = pd.concat([train_iden, test_iden], axis=0).set_index(COLUMN_ID)
# data.set_index(COLUMN_ID, inplace=True)
viz = data.join(iden, how='inner')
viz.to_csv('../eda/eda_start_date_by_identity.csv', index=True)

In [ ]:
# Identity
indicator = 'TransactionAmt'
data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
data['month'] = data['datetime'].map(lambda x: x.month)
data['date'] = data['datetime'].map(lambda x: x.date())
key_cols = [col for col in data.columns if not col.count('Trans') and col != 'datetime' and not col.count('dist')]
viz = data.groupby(key_cols)[indicator].sum()
viz = viz.to_frame(indicator + '_sum').reset_index()
viz.to_csv('../eda/eda_start_date_by_some_category.csv', index=True)